In [1]:
import xarray as xr
from glob import glob
import matplotlib.pyplot as plt
import xroms
# from cartopy import geodesic  # FS only way to get this module
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import numpy as np
import cmocean.cm as cmo

In [2]:
def save_model_at_mooring(var, filename):
    data = var.data
    time = var.ocean_time.data
    depth = var.z_rho.mean('ocean_time').data
    nc = xr.Dataset(
        data_vars=dict(
            temp=(["time", "depth"], data)
        ),
        coords=dict(
            time=(["time"], time),
            depth=(["depth"], depth),
        ),
        attrs=dict(description="EAC_ROMS V3 at "+str(var)),
    )
    nc.to_netcdf(filename + '.nc')
    
#----------------------------------------------------------------
# 
def plot_moor_pos(v3, lon_obs, lat_obs, lon_best, isobath, titlestring):
    dl = .3
    box = (v3.lon_rho > lon_obs - dl) & (v3.lon_rho < lon_obs + dl) & (v3.lat_rho > lat_obs - dl) & (v3.lat_rho < lat_obs + dl)
    dss = v3.where(box, drop=True).h.mean('ocean_time')
    vmin = dss.min().values
    vmax = dss.max().values
    dss.plot(x='lon_rho', y='lat_rho', vmax=200, cmap=cmo.deep)
    dss.plot.contour(x='lon_rho', y='lat_rho', levels=[isobath], colors='k', linestyles='--')
    plt.scatter(lon_obs, lat_obs, s=50, c = 'k', edgecolor='r', label = 'mooring location')
    plt.scatter(lon_best, lat_obs, s=50, c = 'k', edgecolor='w',label = 'validation location')
    plt.xlim(lon_obs - dl, lon_obs + dl)
    plt.ylim(lat_obs - dl, lat_obs + dl)
    plt.legend()
    plt.title(titlestring)


In [3]:
from distributed import Client, progress, LocalCluster
import socket
client = Client(service_kwargs={'dashboard': {'prefix': f'/node/{socket.gethostname()}/8787'}})

/g/data/fu5/deg581/miniconda3/envs/devel3.9/lib/python3.9/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 33265 instead
  warnings.warn(


In [ ]:
def global_vars():
    # A basic chunk choice
    chunks = {'ocean_time': 10}
    glb_files = glob('/srv/scratch/z3533156/26year_BRAN2020/outer_avg_*.nc')
    ds = xr.open_mfdataset(glb_files, chunks=chunks)
    print('global_vars: OK!')
    return ds

v3 = global_vars()

#### Open mooring datasets using THREDDS (FS will include moorings for all the sections - QLD/NSW/VIC)

In [ ]:
# url: http://thredds.aodn.org.au/thredds/catalog/IMOS/ANMN/NSW/catalog.html
ch100  = xr.open_dataset('http://thredds.aodn.org.au/thredds/dodsC/IMOS/ANMN/NSW/CH100/gridded_timeseries/IMOS_ANMN-NSW_TZ_20090815_CH100_FV02_TEMP-gridded-timeseries_END-20220324_C-20220622.nc')
ch70   = xr.open_dataset('http://thredds.aodn.org.au/thredds/dodsC/IMOS/ANMN/NSW/CH070/gridded_timeseries/IMOS_ANMN-NSW_TZ_20090815_CH070_FV02_TEMP-gridded-timeseries_END-20220324_C-20220622.nc')
syd100 = xr.open_dataset('http://thredds.aodn.org.au/thredds/dodsC/IMOS/ANMN/NSW/SYD100/gridded_timeseries/IMOS_ANMN-NSW_TZ_20080625_SYD100_FV02_TEMP-gridded-timeseries_END-20220426_C-20220622.nc')
syd140 = xr.open_dataset('http://thredds.aodn.org.au/thredds/dodsC/IMOS/ANMN/NSW/SYD140/gridded_timeseries/IMOS_ANMN-NSW_TZ_20080625_SYD140_FV02_TEMP-gridded-timeseries_END-20220504_C-20220622.nc')
bmp120 = xr.open_dataset('http://thredds.aodn.org.au/thredds/dodsC/IMOS/ANMN/NSW/BMP120/gridded_timeseries/IMOS_ANMN-NSW_TZ_20110329_BMP120_FV02_TEMP-gridded-timeseries_END-20220516_C-20220622.nc')
bmp70  = xr.open_dataset('http://thredds.aodn.org.au/thredds/dodsC/IMOS/ANMN/NSW/BMP070/gridded_timeseries/IMOS_ANMN-NSW_TZ_20141118_BMP070_FV02_TEMP-gridded-timeseries_END-20220516_C-20220622.nc')

#### Make depth axis negative

In [ ]:
ch100['DEPTH_1']  = ch100.DEPTH * -1
ch70['DEPTH_1']   = ch70.DEPTH * -1
syd100['DEPTH_1'] = syd100.DEPTH * -1
syd140['DEPTH_1'] = syd140.DEPTH * -1
bmp120['DEPTH_1'] = bmp120.DEPTH * -1
bmp70['DEPTH_1']  = bmp120.DEPTH * -1

In [ ]:
def argsel2d(lons, lats, lon0, lat0):
    """Find the indices of coordinate pair closest to another point.
    Inputs
    ------
    lons: DataArray, ndarray, list
        Longitudes of points to search through for closest point.
    lats: DataArray, ndarray, list
        Latitudes of points to search through for closest point.
    lon0: float, int
        Longitude of comparison point.
    lat0: float, int
        Latitude of comparison point.
    Returns
    -------
    Index or indices of location in coordinate pairs made up of lons, lats
    that is closest to location lon0, lat0. Number of dimensions of
    returned indices will correspond to the shape of input lons.
    Notes
    -----
    This function uses Great Circle distance to calculate distances assuming
    longitudes and latitudes as point coordinates. Uses cartopy function
    `Geodesic`: https://scitools.org.uk/cartopy/docs/latest/cartopy/geodesic.html
    If searching for the closest grid node to a lon/lat location, be sure to
    use the correct horizontal grid (rho, u, v, or psi). This is accounted for
    if this function is used through the accessor.
    Example usage
    -------------
    >>> xroms.argsel2d(ds.lon_rho, ds.lat_rho, -96, 27)
    """
    # input lons and lats can be multidimensional and might be DataArrays or lists
    pts = list(zip(*(np.asarray(lons).flatten(), np.asarray(lats).flatten())))
    endpts = list(zip(*(np.asarray(lon0).flatten(), np.asarray(lat0).flatten())))

    G = cartopy.geodesic.Geodesic()  # set up class
    dist = np.asarray(G.inverse(pts, endpts)[:, 0])  # select distances specifically
    iclosest = abs(np.asarray(dist)).argmin()  # find indices of closest point
    # return index or indices in input array shape
    inds = np.unravel_index(iclosest, np.asarray(lons).shape)
    return inds

def sel2d(var, lons, lats, lon0, lat0):
    """Find the value of the var at closest location to lon0,lat0.
    Inputs
    ------
    var: DataArray, ndarray
        Variable to operate on.
    lons: DataArray, ndarray, list
        Longitudes of points to search through for closest point.
    lats: DataArray, ndarray, list
        Latitudes of points to search through for closest point.
    lon0: float, int
        Longitude of comparison point.
    lat0: float, int
        Latitude of comparison point.
    Returns
    -------
    Value in var of location in coordinate pairs made up of lons, lats
    that is closest to location lon0, lat0. If var has other
    dimensions, they are brought along.
    Notes
    -----
    This function uses Great Circle distance to calculate distances assuming
    longitudes and latitudes as point coordinates. Uses cartopy function
    `Geodesic`: https://scitools.org.uk/cartopy/docs/latest/cartopy/geodesic.html
    If searching for the closest grid node to a lon/lat location, be sure to
    use the correct horizontal grid (rho, u, v, or psi). This is accounted for
    if this function is used through the accessor.
    This is meant to be used by the accessor to conveniently wrap
    `argsel2d`.
    Example usage
    -------------
    >>> xroms.sel2d(ds.temp, ds.lon_rho, ds.lat_rho, -96, 27)
    """
    assert isinstance(var, xr.DataArray), "Input a DataArray"
    inds = argsel2d(lons, lats, lon0, lat0)
#     return var.cf.isel(Y=inds[0], X=inds[1])   #FS
    return var.cf.isel(eta_rho=inds[0], xi_rho=inds[1])     #FS



In [ ]:
lon = v3.lon_rho.values
lat = v3.lat_rho.values

In [ ]:
def find_best_model_depth(moor_lon, moor_lat, moor_depth, model_lon, nm):
    print('Doing for %s' % nm)
    moor_deepest = np.min(moor_depth)
    resol = np.min(np.diff(lon))
    st = moor_lon - (resol*10)
    nd = moor_lon + (resol*10)
    print('This is the original mooring longitude: %s' % moor_lon.data)
    print('This is the starting range mooring longitude: %s' % st.data)
    print('This is the ending range mooring longitude: %s' % nd.data)
    seed_array = np.linspace(st, nd, 20)
    depth_diff = list()
    sto_model_idx = list()
    for tlon in seed_array:
        model_depth = sel2d(v3.h, lon, lat, tlon, moor_lat)[0]
        depth_diff.append(abs(abs(moor_deepest) - model_depth.values))
        sto_model_idx.append(model_depth.values)
    idx = np.argsort(depth_diff)[0]
    print('For the longitude %s, the depth difference between mooring and model is %s meters' % (seed_array[idx], np.round(depth_diff[idx].data, 2)))
    print('Mooring depth is %s and model depth is %s meters.' % (abs(moor_deepest.data), sto_model_idx[idx]))
    return seed_array[idx]

## Coffs coordinates

In [ ]:
ch100_lon, ch100_lat = ch100.LONGITUDE, ch100.LATITUDE
ch100_lon_best = find_best_model_depth(ch100_lon, ch100_lat, ch100.DEPTH_1, lon, 'CH100')

In [ ]:
ch70_lon, ch70_lat = ch70.LONGITUDE, ch70.LATITUDE
ch70_lon_best = find_best_model_depth(ch70_lon, ch70_lat, ch70.DEPTH_1, lon, 'CH70')

### Sydney Coordinates

In [ ]:
syd100_lon, syd100_lat = syd100.LONGITUDE, syd100.LATITUDE
syd100_lon_best = find_best_model_depth(syd100_lon, syd100_lat, syd100.DEPTH_1, lon, 'SYD100')

In [ ]:
syd140_lon, syd140_lat = syd140.LONGITUDE, syd140.LATITUDE
syd140_lon_best = find_best_model_depth(syd140_lon, syd140_lat, syd140.DEPTH_1, lon, 'SYD140')

### Narooma coordinates

In [ ]:
# dif from Neil code> 131.58928345 - 120 (11.589), difference bigger than mine...
bmp120_lon, bmp120_lat = bmp120.LONGITUDE, bmp120.LATITUDE
bmp120_lon_best = find_best_model_depth(bmp120_lon, bmp120_lat, bmp120.DEPTH_1, lon, 'BMP120')

In [ ]:
bmp70_lon, bmp70_lat = bmp70.LONGITUDE, bmp70.LATITUDE
bmp70_lon_best = find_best_model_depth(bmp70_lon, bmp70_lat, bmp70.DEPTH_1, lon, 'BMP70')

### Plot locations for sanity check:

In [ ]:
fig = plt.figure(figsize=(16,5))
plt.subplot(2, 3, 1)
plot_moor_pos(v3, ch100_lon, ch100_lat, ch100_lon_best, 100, 'CH100 mooring locations')
plt.subplot(2, 3, 2)
plot_moor_pos(v3, ch70_lon, ch70_lat, ch70_lon_best, 100, 'CH70 mooring locations')
plt.subplot(2, 3, 3)
plot_moor_pos(v3, syd100_lon, syd100_lat, syd100_lon_best, 100, 'SYD100 mooring locations')
plt.subplot(2, 3, 4)
plot_moor_pos(v3, syd140_lon, syd140_lat, syd140_lon_best, 100, 'SYD140 mooring locations')
plt.subplot(2, 3, 5)
plot_moor_pos(v3, bmp120_lon, bmp120_lat, bmp120_lon_best, 100, 'BMP120 mooring locations')
plt.subplot(2, 3, 6)
plot_moor_pos(v3, bmp70_lon, bmp70_lat, bmp70_lon_best, 100, 'BMP70 mooring locations')
plt.tight_layout()
plt.savefig('validation_point.png', format="png", bbox_inches='tight', pad_inches=0.1, dpi=100)
plt.close()

In [ ]:
def sigma_depth_calc(ds):
    '''Sigma layers depth calculation'''
    Zo_rho = (ds.hc * ds.s_rho + ds.Cs_r * ds.h) / (ds.hc + ds.h)
    z_rho = ds.zeta + (ds.zeta + ds.h) * Zo_rho
    ds.coords["z_rho"] = z_rho
    print('sigma_depth_calc: OK!')
    return ds

v3 = sigma_depth_calc(v3)

In [ ]:
v3_ch100  = sel2d(v3.temp, lon, lat, ch100_lon_best, ch100_lat).sel(ocean_time=slice(ch100.TIME[0], v3.ocean_time[-1])).load()
v3_ch70   = sel2d(v3.temp, lon, lat, ch70_lon_best, ch70_lat).sel(ocean_time=slice(ch70.TIME[0], v3.ocean_time[-1])).load()
v3_syd100 = sel2d(v3.temp, lon, lat, syd100_lon_best, syd100_lat).sel(ocean_time=slice(syd100.TIME[0], v3.ocean_time[-1])).load()
v3_syd140 = sel2d(v3.temp, lon, lat, syd140_lon_best, syd140_lat).sel(ocean_time=slice(syd140.TIME[0], v3.ocean_time[-1])).load()
v3_bmp120 = sel2d(v3.temp, lon, lat, bmp120_lon_best, bmp120_lat).sel(ocean_time=slice(bmp120.TIME[0], v3.ocean_time[-1])).load()
v3_bmp70  = sel2d(v3.temp, lon, lat, bmp70_lon_best, bmp70_lat).sel(ocean_time=slice(bmp70.TIME[0], v3.ocean_time[-1])).load()


##### Save out temperature timeseries from model runs

In [ ]:
save_model_at_mooring(v3_ch100, '/home/z5392640/work/phd_unsw/katana/roms_validation/v3_ch100' )
save_model_at_mooring(v3_ch70, '/home/z5392640/work/phd_unsw/katana/roms_validation/v3_ch70' )
save_model_at_mooring(v3_syd100, '/home/z5392640/work/phd_unsw/katana/roms_validation/v3_syd100' )
save_model_at_mooring(v3_syd140, '/home/z5392640/work/phd_unsw/katana/roms_validation/v3_syd140' )
save_model_at_mooring(v3_bmp120, '/home/z5392640/work/phd_unsw/katana/roms_validation/v3_bmp120' )
save_model_at_mooring(v3_bmp70, '/home/z5392640/work/phd_unsw/katana/roms_validation/v3_bmp70' )

### Plot time-mean profiles to compare

In [ ]:
v3_time = v3_ch100.mean('ocean_time')
v3_depth = v3_ch100.z_rho.mean('ocean_time')
ch100_cut = ch100.TEMP.sel(TIME=slice(ch100.TIME[0], v3.ocean_time[-1])).mean('TIME')
plt.plot(v3_time, v3_depth, color='orange', label='v3')
plt.scatter(ch100_cut, ch100.DEPTH_1, color='k', label='Obs')
plt.title('CH100 ROMS V3')
plt.legend()
plt.savefig('v3_ch100_temp.png',format="png",bbox_inches='tight',pad_inches=0.1, dpi=100)

In [ ]:
v3_time = v3_ch70.mean('ocean_time')
v3_depth = v3_ch70.z_rho.mean('ocean_time')
ch70_cut = ch70.TEMP.sel(TIME=slice(ch70.TIME[0], v3.ocean_time[-1])).mean('TIME')
plt.plot(v3_time, v3_depth, color='orange', label='v3')
plt.scatter(ch70_cut, ch70.DEPTH_1, color='k', label='Obs')
plt.title('CH70 ROMS V3')
plt.legend()
plt.savefig('v3_ch70_temp.png', format="png",bbox_inches='tight',pad_inches=0.1, dpi=100)

In [ ]:
v3_time = v3_syd100.mean('ocean_time')
v3_depth = v3_syd100.z_rho.mean('ocean_time')
syd100_cut = syd100.TEMP.sel(TIME=slice(syd100.TIME[0], v3.ocean_time[-1])).mean('TIME')
plt.plot(v3_time, v3_depth, color='orange', label='v3')
plt.scatter(syd100_cut, syd100.DEPTH_1, color='k', label='Obs')
plt.title('SYD100 ROMS V3')
plt.legend()
plt.savefig('v3_syd100_temp.png',format="png",bbox_inches='tight',pad_inches=0.1, dpi=100)

In [ ]:
v3_time = v3_syd140.mean('ocean_time')
v3_depth = v3_syd140.z_rho.mean('ocean_time')
syd140_cut = syd140.TEMP.sel(TIME=slice(syd140.TIME[0], v3.ocean_time[-1])).mean('TIME')
plt.plot(v3_time, v3_depth, color='orange', label='v3')
plt.scatter(syd140_cut, syd140.DEPTH_1, color='k', label='Obs')
plt.title('SYD140 ROMS V3')
plt.legend()
plt.savefig('v3_syd140_temp.png',format="png",bbox_inches='tight',pad_inches=0.1, dpi=100)

In [ ]:
v3_time = v3_bmp120.mean('ocean_time')
v3_depth = v3_bmp120.z_rho.mean('ocean_time')
bmp120_cut = bmp120.TEMP.sel(TIME=slice(bmp120.TIME[0], v3.ocean_time[-1])).mean('TIME')
plt.plot(v3_time, v3_depth, color='orange', label='v3')
plt.scatter(bmp120_cut, bmp120.DEPTH_1, color='k', label='Obs')
plt.title('BMP120 ROMS V3')
plt.legend()
plt.savefig('v3_bmp120_temp.png',format="png",bbox_inches='tight',pad_inches=0.1, dpi=100)

In [ ]:
v3_time = v3_bmp70.mean('ocean_time')
v3_depth = v3_bmp70.z_rho.mean('ocean_time')
bmp70_cut = bmp70.TEMP.sel(TIME=slice(bmp70.TIME[0], v3.ocean_time[-1])).mean('TIME')
plt.plot(v3_time, v3_depth, color='orange', label='v3')
plt.scatter(bmp70_cut, bmp70.DEPTH_1, color='k', label='Obs')
plt.title('BMP70 ROMS V3')
plt.legend()
plt.savefig('v3_bmp70_temp.png',format="png",bbox_inches='tight',pad_inches=0.1, dpi=100)